<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [0]:
import numpy as np
from copy import copy
import heapq

In [0]:
# Import any packages you need here


# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self, state, heuristic = None, parent = None, moves=0):
      #Storing the state as a list
      self.state = state
      
      #I use the numpy library so I store one numpy version of the state
      self.numpystate = np.array(state)
      
      #Storing the dimension
      self.n = len(state)

      #Storing the parent node
      self.parent = parent
      
      #Number of steps so far to reach the node
      self.moves = moves
      
      #Which heuristic function is being used
      self.heuristic_function = heuristic
      
      #The output of the heuristic function
      self.heuristic_value = self.heuristic_function(self.state)
      
      #Summing the heuristic value with the steps so far for f()
      self.f_value = self.moves + self.heuristic_value

      #Id to compare visited nodes. Since each configuration is unique I flatten the array and store it as a string
      self.puzzle_id = "".join(map(str, self.numpystate.flatten()))
      
      #Index of zero in the flattened version. Used to find the possible moves
      self.zero_index = int(np.where(self.numpystate.flatten() == 0)[0][0])

    
    #string representation. Looping and creating a string of N*N dimensions
    def __str__(self):
      rows = []
      for i in range(self.n):
        row = ''
        for j in range(self.n):
          row = row + (str(self.state[i][j])) + ' '
        rows.append(row)
      return '\n'.join(rows)

    #Eq function to 
    #def __eq__(self, node):
      #return self.puzzle_id == node.puzzle_id

    #Used to make comparisons based on the f() value, so they can be pushed into the priority queue
    def __lt__(self, other):
      return self.f_value < other.f_value

    #Function to return the parent node
    def get_parent(self):
      return self.parent

    #Function that chacks id the state is a valid instance of the puzzle
    def check_if_valid(self):
      #Since in the given representation, we need each numer from 0 to n in n by n nested lists
      #we check if the flattened list when sorted is equal to the range between 0 to n
      if np.all((sorted(self.numpystate.flatten()) == np.arange(self.n ** 2)) == 1):
        return True
      else:
        return False
    
    #The function to compute the potential successor states
    #There are 4 moves that a 0 cell can do. Based on what edge it's on it cannot do some of the moves
    #For example if it is on the right edge it cannot move to right. Down below I have a clause for each case
    #If the 0 index passes it a move is granted to it. For each case I found a representation to indicate
    #the position of the 0 by using mod and floor divisions

    def get_successors(self):
      
      #0 not on the right edge
      if self.zero_index % self.n != -1 % self.n:
        #A new state is created
        new_state = copy(self.numpystate.flatten())

        #Zero and the element right to it are swapped
        new_state[self.zero_index], new_state[self.zero_index+1] = new_state[self.zero_index+1], new_state[self.zero_index]
        
        #Turned into a regular list
        new_state = new_state.reshape(self.n, self.n).tolist()
        
        #New node is instansiated
        yield PuzzleNode(state = new_state, parent = self, moves=self.moves+1, heuristic=self.heuristic_function) 

      #0 not on the left edge, we move 0 to left, everything else same as above
      if self.zero_index % self.n != 0:
        new_state = copy(self.numpystate.flatten())
        new_state[self.zero_index], new_state[self.zero_index-1] = new_state[self.zero_index-1], new_state[self.zero_index]
        new_state = new_state.reshape(self.n, self.n).tolist()
        yield PuzzleNode(state = new_state, parent = self, moves=self.moves+1, heuristic=self.heuristic_function)

      #0 not on the top edge, we move 0 upwards, everything else same as above
      if self.zero_index//self.n != 0:
        new_state = copy(self.numpystate.flatten())
        new_state[self.zero_index], new_state[self.zero_index-self.n] = new_state[self.zero_index-self.n], new_state[self.zero_index]
        new_state = new_state.reshape(self.n, self.n).tolist()
        yield PuzzleNode(state = new_state, parent = self, moves=self.moves+1, heuristic=self.heuristic_function)

      #0 not on the bottom edge, we move 0 downwards, everything else same as above
      if self.zero_index//self.n != self.n-1:
        new_state = copy(self.numpystate.flatten())
        new_state[self.zero_index], new_state[self.zero_index+self.n] = new_state[self.zero_index+self.n], new_state[self.zero_index]
        new_state = new_state.reshape(self.n, self.n).tolist()
        yield PuzzleNode(state = new_state, parent = self, moves=self.moves+1, heuristic=self.heuristic_function)

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [0]:
# Add any additional code here (e.g. for the memoization extension)
# YOUR CODE HERE (OPTIONAL)

#Creating a frontier class. A heap data structure is used for the priority queue
class Frontier:
  
  #Initializing 
  def __init__(self):
    #Includes a list for the queue
    self.queue = []
    
    #Max size is recorded
    self.max_size = 0
  
  #Function that add nodes.
  def add_node(self, node):
    #Heappush the node into the list. They are compared based on the f() value as specified inside the NodePuzzle class
    heapq.heappush(self.queue, node)

    #If a new maximum size is reached it is updated
    if self.length() > self.max_size:
      self.max_size = self.length()

  #Popping from the queue
  def pop(self):
    #Uses heappop
    return heapq.heappop(self.queue)

  #Function to return the length
  def length(self):
    return len(self.queue)


# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    #np.arange(len(state)**2).reshape(len(state), len(state)) gives a solved puzzle instance in the given size
    #We subtract the state from it. A subtraction yields 0 for the matching elements
    #We subtract it from the total number of cells and then subtract 1 from it

    return len(state)**2 - np.sum((np.arange(len(state)**2).reshape(len(state), len(state)) - np.array(state)) == 0) - 1

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    #initial manhattan distinace
    h=0
    #looping for all cells
    for i in range(len(state)):
      for j in range(len(state)):
        #Not taking 0 into account
        if state[i][j] != 0:
          #Summing the amount the x and y coordinates of a cell should move
          h += abs(i - state[i][j]//len(state)) + abs(j - state[i][j]%len(state))

    return h      
    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [0]:
# Import any packages or define any helper functions you need here

# YOUR CODE HERE (OPTIONAL)
# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """

    # YOUR CODE HERE
    #First creating a node instance for the initial state
    initial_node = PuzzleNode(state, heuristic)
    
    #Checking for validness first
    if not initial_node.check_if_valid():
      return 0, 0, 0, None, -1

    #Creating a list to store the visited nodes
    visited_nodes = []
    
    #Creating a frontier object
    frontier = Frontier()
    
    #Creating the goal state as puzzle node instance
    goal_state = PuzzleNode(np.arange(initial_node.n**2).reshape(initial_node.n, initial_node.n).tolist(), heuristic)

    #boolean for the while loop
    solved = False

    #Checking if we are already at the goal state
    if initial_node.puzzle_id == goal_state.puzzle_id:
      solved = True
    #If we are not the initial state is added to the frontier. This is done for practicality so it will work with the while loop.
    else:
      frontier.add_node(initial_node)

    #Starting the while loop
    while solved == False:
      
      #If we visit all the reachable nodes and cannot find a solution then we return -2
      #This theoretically should work. But there are too many nodes to visit before this halts
      if frontier.length() == 0:
        return current_node.moves, len(visited_nodes), frontier.max_size, None, -2

      #Popping from the frontier
      current_node = frontier.pop()
      
      #Appending the current node into the visited nodes
      visited_nodes.append(current_node.puzzle_id)

      #Checking if the current node is the goal state
      if current_node.puzzle_id == goal_state.puzzle_id:
        solved = True

      #Expanding the tree to find the successors
      for node in current_node.get_successors():
        #If a node is visited it isn't added to the frontier
        if node.puzzle_id not in visited_nodes:
          frontier.add_node(node)


    #Counting the steps when the while loop executes
    steps = current_node.moves

    #A list to store the optimal path
    optimal_path = []

    #Runs and traces the path until we return back to the initial node
    while current_node != initial_node:
      #Inserting optimal path to the left side
      optimal_path.insert(0, current_node.state)
      #Current node is replaced with its parent
      current_node = current_node.parent

    #The initla node is also added to the optimal path
    optimal_path.insert(0, initial_node.state)


    #Returns the required parameters
    return steps, len(visited_nodes), frontier.max_size, optimal_path, _

    

      


<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [0]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [0]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


Hi Professor! The last test cell runs. However, it takes a long time. I am writing you this note so you wouldn't think it is an endless loop. In my computer it takes around 9 minutes to run. I tried to make it more efficient but this was as fast as it got. However, if you run them seperately you will see it solves each of the initial configurations and gives the correct output.

In [0]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [0]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [0]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [0]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [0]:
## Memoization test - will be carried out after submission